# SVD Classifier

revelvent infor has come so far from https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 
import time
import random
#!pip install ProgressBar
from progressbar import ProgressBar
import math
#!{sys.executable} -m pip install ProgressBar
#!pip install ProgressBar
from random import randrange
from copy import copy, deepcopy

import glob
import json

#svd stuff
from scipy.linalg import svd
from numpy.linalg import pinv


In [3]:
#load the titles and wheater the book is alive or dead into two seperate arrays
def loadResult():
    test_labels = []
    test_values = []
    with open('table-of-contents.json', encoding="utf8") as f:
        data = json.load(f)
        for books in data:
            for book in data[books]:
                test_labels.append(book['title'])
                test_values.append(book['dead'])
                
    return test_labels, test_values

#get the data from the data output file
def loadData():
    final_data = []
    with open('dataOutput2.json', encoding="utf8") as f:
        data = json.load(f)
        for index in data: 
            final_data.append(data[index]['chapData'][:-2])
    return final_data

In [4]:

                

title_labels, dead_labels = loadResult()
train = loadData()
print(len(train))
## Load the training set
train_data= np.array(train[:15705])
train_labels = np.array(dead_labels[:15705])
train_titles = np.array(title_labels[:15705])


## Load the testing set

test_data= np.array(train[-1000:])
test_labels = np.array(dead_labels[-1000:])
test_titles = np.array(title_labels[-1000:])


## Print out their dimensions
print("Training dataset dimensions: ", np.shape(train_data))
print("Number of training labels: ", len(train_labels))
print("Testing dataset dimensions: ", np.shape(test_data))
print("Number of testing labels: ", len(test_labels))



16705
Training dataset dimensions:  (15705, 69)
Number of training labels:  15705
Testing dataset dimensions:  (1000, 69)
Number of testing labels:  1000


For the next two items we are attempting to recreating the element. A should equal B


calculate pseudo inverse manually and compare to the built in version

In [5]:
#A= np.array([[0.1, 0.2],[0.3, 0.4],[0.5, 0.6],[0.7, 0.8]])
A=np.array(train_data[:])
print(A)
# calculate pseudoinverse
B = pinv(A)
print(B)

[[   0    0    0 ...  849    2    0]
 [   0    0    0 ...  336    0    0]
 [   0    0    0 ...  417   70   20]
 ...
 [   1    0    0 ... 3698  361   96]
 [   0    0    0 ...  142   13    5]
 [   0    0    0 ...  144    1    1]]
[[ 1.10539048e-06 -7.78396633e-05 -1.91528908e-05 ...  4.97438820e-04
  -2.80535040e-04 -8.64017032e-05]
 [-3.08622659e-05  4.24987368e-05 -2.50498934e-04 ... -1.72169825e-04
  -2.19158417e-04 -1.35247758e-04]
 [-2.39926333e-05 -3.70039538e-05 -9.23720359e-05 ... -1.09323794e-05
  -2.75439789e-04 -1.88112373e-04]
 ...
 [ 2.04634607e-09 -7.12583286e-09 -1.13517115e-08 ...  3.85638986e-09
   9.53456417e-09 -2.63637089e-08]
 [ 2.52959025e-08 -9.56609454e-08 -1.84438388e-07 ...  3.44069819e-07
  -1.38855558e-07 -1.34649674e-07]
 [-1.43451377e-07  3.33387678e-07  4.42646942e-07 ... -6.35014565e-07
   2.39143595e-07  5.91673585e-07]]


In [6]:
print(A)
# calculate svd
U, s, VT = svd(A)
# reciprocals of s
d = 1.0 / s
# create m x n D matrix
D = np.zeros(A.shape)
# populate D with n x n diagonal matrix
D[:A.shape[1], :A.shape[1]] = np.diag(d)
# calculate pseudoinverse
B = VT.T.dot(D.T).dot(U.T)
print(B)


[[   0    0    0 ...  849    2    0]
 [   0    0    0 ...  336    0    0]
 [   0    0    0 ...  417   70   20]
 ...
 [   1    0    0 ... 3698  361   96]
 [   0    0    0 ...  142   13    5]
 [   0    0    0 ...  144    1    1]]
[[ 1.10539048e-06 -7.78396633e-05 -1.91528908e-05 ...  4.97438820e-04
  -2.80535040e-04 -8.64017032e-05]
 [-3.08626433e-05  4.24987842e-05 -2.50498858e-04 ... -1.72169833e-04
  -2.19158502e-04 -1.35247772e-04]
 [-2.39902283e-05 -3.70042778e-05 -9.23725910e-05 ... -1.09323130e-05
  -2.75438837e-04 -1.88112272e-04]
 ...
 [ 2.04636440e-09 -7.12583510e-09 -1.13517150e-08 ...  3.85639025e-09
   9.53456725e-09 -2.63637083e-08]
 [ 2.52958416e-08 -9.56609392e-08 -1.84438381e-07 ...  3.44069819e-07
  -1.38855544e-07 -1.34649675e-07]
 [-1.43451749e-07  3.33387730e-07  4.42647034e-07 ... -6.35014576e-07
   2.39143413e-07  5.91673569e-07]]


In [13]:

true = 0

for i in range(len(train_data)):
    x = train_data[i]
    predict = np.sum(x.dot(B))
    
    #find the error
    if (train_labels[i] == False) and (predict<1):
        true +=1
    elif (train_labels[i] == True) and (predict>1):
        true+=1
    
   
    
    #these are  the results from the svd classification
    #print(predict, end = ": ")
    #print(train_labels[i])
    
    #calculate the error
    
print('eeror rate: ', 1-(true/len(train_data)))

eeror rate:  0.4213944603629417
